In [1]:
import sqlite3

In [2]:
# create table ANNOTATION
conn = sqlite3.connect('ANNOTATION2.db')
c = conn.cursor()
cmd_creat_table = '''CREATE TABLE ANNOTATION (
ID chr NOT NULL, 
species chr NOT NULL, 
chromosome chr NOT NULL, 
start INT NOT NULL, 
end INT NOT NULL, 
strand INT NOT NULL,  
GenBank chr, 
PomBase chr, 
Symbol chr, 
prod chr, 
PRIMARY KEY (ID, species));
'''
c.execute(cmd_creat_table)
conn.commit()

In [3]:
def insertAnno(conn, ID = None, species = None, chromosome = None, start = None, 
               end = None, strand = None, 
               GenBank = None, PomBase = None, Symbol = None, prod = None):
    assert all([ID, species, chromosome, start, end, strand]), "NOT NULL fields happen to be None"
    
    fields, values = [], []
    for k, v in locals().items():
        if v and (k not in {'conn', 'fields', 'values'}):
            fields.append(k)
            if type(v) == str:
                values.append('"'+v+'"')
            else:
                values.append(str(v))
    fields = ', '.join(fields)
    values = ', '.join(values)

    #speciesMap = {'P': 'S_pombe.gff.db'}
    #conn = sqlite3.connect(speciesMap[species])
    c = conn.cursor()
    cmd_insert_row = 'INSERT INTO ANNOTATION ({}) VALUES ({});'.format(fields, values)
    c.execute(cmd_insert_row)
    conn.commit()
#insertAnno(conn, species='C', chromosome='chromosome1', start=1, end=300, strand=1, ID='cds1')
#insertAnno(conn, species='P', chromosome='chromosome1', start=1, end=300, strand=1, ID='cds2')

In [4]:
def gff2sql(file_name, species, conn):
    R = {'ID': None, 'species': species, 'chromosome': None, 'pos': [], 
         'strand': None, 'GenBank': None, 'PomBase': set(), 'Symbol': None}
    lastID = None
    with open(file_name, 'r') as GFF:
        for line in GFF:
            line = line.strip('\n')
            tokens = line.split('\t')
            chromosome = tokens[0]
            pos = (tokens[3], tokens[4])
            strand = {'+': 1, '-': -1}[tokens[6]]
            info = tokens[8]
            ID, GenBank, PomBase, Symbol = None, None, set(), None
            for i in info.split(';'):
                if i.startswith('ID=') and ID == None:
                    ID = i[3:]
                elif i.startswith('Name=') and GenBank == None:
                    GenBank = i[5:]
                elif i.startswith('gene=') and Symbol == None:
                    Symbol = i[5:]
                elif i.startswith('Dbxref='):
                    for j in i.split(','):
                        if j.startswith('PomBase:'):
                            PomBase.add(j.split(':')[1])
                elif i.startswith('product='):
                    prod = i[8:]
            if ID != lastID and lastID != None:
                # Push into the SQL table
                if R['strand'] == 1:
                    start, end = R['pos'][0][0], R['pos'][-1][-1]
                else:
                    start, end = R['pos'][-1][0], R['pos'][0][1]
                sysid = ';'.join(R['PomBase'])
                print(sysid)
                if sysid == '':
                    sysid = None
                insertAnno(conn, ID = R['ID'], species = R['species'], 
                           chromosome = R['chromosome'], start = int(start), 
                           end = int(end), strand = R['strand'], GenBank = R['GenBank'], 
                           PomBase = sysid, Symbol = R['Symbol'], prod = R['prod'])
                R['pos'] = []
                R['PomBase'] = set()
            R['ID'], R['chromosome'], R['strand'], R['GenBank'], R['Symbol'] = ID, chromosome, strand, GenBank, Symbol
            R['pos'].append(pos)
            R['PomBase'] = R['PomBase'].union(PomBase)
            R['prod'] = prod
                
            lastID = ID
    
    start, end = R['pos'][0][0], R['pos'][-1][-1]
    sysid = ';'.join(R['PomBase'])
    print(sysid)
    if sysid == '':
        sysid = None
    insertAnno(conn, ID = R['ID'], species = R['species'], 
               chromosome = R['chromosome'], start = int(start), 
               end = int(end), strand = R['strand'], GenBank = R['GenBank'], 
               PomBase = sysid, Symbol = R['Symbol'], prod = R['prod'])
    
    GFF.close()
gff2sql('CDS_only.gff', 'C', conn)

In [5]:
def showAnno(conn):
    c = conn.cursor()
    result = c.execute('SELECT * FROM ANNOTATION;')
    for row in result.fetchall():
        print(row)
showAnno(conn)

('cds0', 'C', 'NW_013185619.1', 2433, 3218, 1, 'XP_013016566.1', None, None, 'versicolorin reductase')
('cds1', 'C', 'NW_013185619.1', 3938, 4096, -1, 'XP_013016567.1', None, None, 'hypothetical protein')
('cds2', 'C', 'NW_013185619.1', 5090, 6046, -1, 'XP_013016568.1', None, None, 'iron/ascorbate oxidoreductase')
('cds3', 'C', 'NW_013185619.1', 7610, 8400, 1, 'XP_013016569.1', None, None, 'short-chain type dehydrogenase/reductase')
('cds4', 'C', 'NW_013185619.1', 9106, 10146, 1, 'XP_013016570.1', None, None, '2-dehydropantoate 2-reductase')
('cds5', 'C', 'NW_013185619.1', 10543, 11352, -1, 'XP_013016571.1', None, None, 'UbiE family methyltransferase')
('cds6', 'C', 'NW_013185619.1', 13101, 13868, 1, 'XP_013016572.1', None, None, 'versicolorin reductase')
('cds7', 'C', 'NW_013185619.1', 15675, 16559, 1, 'XP_013016573.1', None, None, 'hypothetical protein')
('cds8', 'C', 'NW_013185619.1', 17333, 19027, -1, 'XP_013016574.1', None, None, 'transcription factor')
('cds9', 'C', 'NW_013185619

('cds1491', 'C', 'NW_013185619.1', 3463311, 3464098, -1, 'XP_013016315.1', None, None, 'TIM23 translocase complex-associated motor subunit Pam17')
('cds1492', 'C', 'NW_013185619.1', 3464409, 3466343, 1, 'XP_013016316.1', None, None, 'U4/U6 X U5 tri-snRNP complex subunit Snu66')
('cds1493', 'C', 'NW_013185619.1', 3466408, 3467664, -1, 'XP_013016317.1', None, None, 'geranylgeranyltransferase II beta subunit Ptb1')
('cds1494', 'C', 'NW_013185619.1', 3467869, 3468493, 1, 'XP_013017857.1', None, None, 'hypothetical protein')
('cds1495', 'C', 'NW_013185619.1', 3468661, 3472289, -1, 'XP_013016318.1', None, None, 'IRE protein kinase Ire1')
('cds1496', 'C', 'NW_013185619.1', 3472696, 3473233, 1, 'XP_013016319.1', None, None, 'hypothetical protein')
('cds1497', 'C', 'NW_013185619.1', 3473861, 3475108, -1, 'XP_013016320.1', None, None, 'cell wall protein Ecm33')
('cds1498', 'C', 'NW_013185619.1', 3476834, 3477514, 1, 'XP_013016321.1', None, None, 'RING-box protein 1')
('cds1499', 'C', 'NW_0131856

('cds2265', 'C', 'NW_013185620.1', 619248, 620622, 1, 'XP_013019274.1', None, None, 'translation release factor eRF1 family protein')
('cds2266', 'C', 'NW_013185620.1', 620885, 622473, -1, 'XP_013019275.1', None, None, 'nucleoporin Nup61')
('cds2267', 'C', 'NW_013185620.1', 622663, 625551, -1, 'XP_013019276.1', None, None, 'isoleucine-tRNA ligase')
('cds2268', 'C', 'NW_013185620.1', 625997, 626347, -1, 'XP_013019277.1', None, None, 'hypothetical protein')
('cds2269', 'C', 'NW_013185620.1', 626512, 627719, -1, 'XP_013019278.1', None, None, 'TRE Xcomplex subunit Tex1')
('cds2270', 'C', 'NW_013185620.1', 627879, 629626, -1, 'XP_013019279.1', None, None, 'CAMK/RAD53 protein kinase Cds1')
('cds2271', 'C', 'NW_013185620.1', 630115, 630882, 1, 'XP_013019280.1', None, None, 'dephospho-CoA kinase')
('cds2272', 'C', 'NW_013185620.1', 631240, 635969, 1, 'XP_013019281.1', None, None, 'ribosome biogenesis protein Urb1')
('cds2273', 'C', 'NW_013185620.1', 636284, 636819, -1, 'XP_013019282.1', None, 

('cds3879', 'C', 'NW_013185621.1', 468810, 469898, 1, 'XP_013020644.1', None, None, 'thioredoxin family protein')
('cds3880', 'C', 'NW_013185621.1', 470163, 472901, -1, 'XP_013020643.1', None, None, 'exocyst complex subunit Sec10')
('cds3881', 'C', 'NW_013185621.1', 473464, 474171, -1, 'XP_013020642.1', None, None, 'hypothetical protein')
('cds3882', 'C', 'NW_013185621.1', 475564, 477327, 1, 'XP_013020641.1', None, None, 'FAD binding protein')
('cds3883', 'C', 'NW_013185621.1', 477540, 478707, -1, 'XP_013020640.1', None, None, 'acyltransferase')
('cds3884', 'C', 'NW_013185621.1', 478992, 480026, 1, 'XP_013020639.1', None, None, 'Sim4 and Mal2 associated protein 2')
('cds3885', 'C', 'NW_013185621.1', 480138, 481476, -1, 'XP_013020638.1', None, None, 'Sir2 family histone deacetylase Hst4')
('cds3886', 'C', 'NW_013185621.1', 482927, 486997, 1, 'XP_013020637.1', None, None, 'ATP-dependent DNA helicase Hrp1')
('cds3887', 'C', 'NW_013185621.1', 487275, 487772, -1, 'XP_013020636.1', None, Non

('cds4896', 'C', 'NW_013185621.1', 2872498, 2874339, 1, 'XP_013020247.1', None, None, 'U3 snoRNP-associated protein Utp3')
('cds4897', 'C', 'NW_013185621.1', 2874894, 2876297, -1, 'XP_013020248.1', None, None, 'Nem1-Spo7 phosphatase complex catalytic subunit Nem1')
('cds4898', 'C', 'NW_013185621.1', 2876912, 2879527, 1, 'XP_013020249.1', None, None, 'RNA polymerase I transcription factor subunit Rrn6')
('cds4899', 'C', 'NW_013185621.1', 2880007, 2882058, 1, 'XP_013020250.1', None, None, 'ATP-dependent DNA helicase Irc3')
('cds4900', 'C', 'NW_013185621.1', 2882158, 2883370, -1, 'XP_013020251.1', None, None, 'tRNA pseudouridine synthase')
('cds4901', 'C', 'NW_013185621.1', 2883575, 2884459, 1, 'XP_013020252.1', None, None, '60S ribosomal protein L5')
('cds4902', 'C', 'NW_013185621.1', 2884969, 2885507, 1, 'XP_013020253.1', None, None, 'hypothetical protein')
('cds4903', 'C', 'NW_013185621.1', 2885629, 2887075, -1, 'XP_013020254.1', None, None, 'NAD/NADH kinase')
('cds4904', 'C', 'NW_0131

In [22]:
# Search the database
def search(conn, keywords):
    cmd = '''SELECT * 
    FROM ANNOTATION
    WHERE PomBase LIKE "%{0}%" OR GenBank LIKE "%{0}%" OR Symbol LIKE "%{0}%";'''.format(keywords)
    c = conn.cursor()
    result = c.execute(cmd)
    for i in result.fetchall():
        print(i)

#conn = sqlite3.connect('S_pombe.gff.db')
search(conn, 'NP_594156.1')

('cds1419', 'P', 'NC_003424.3', 3361547, 3365548, -1, 'NP_594156.1', 'SPAC9.04.1;SPAC26A3.13c.1;SPAC167.08;SPAC167.08.1;SPAC9.04;SPAC26A3.13c', 'Tf2-4')


In [6]:
conn.close()